## CRISP-DM Phase 4. Modelling

In this phase, various modeling techniques are selected and applied, and their parameters are calibrated to optimal values. Typically, there are several techniques for the same data mining problem type. Some techniques have specific requirements on the form of data. Therefore, going back to the data preparation phase is often necessary.

In [1]:
# Dependencies
import sys
sys.path.append('../src')
import utils
import data_processor
import model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pickle as pkl
import warnings 
warnings.filterwarnings('ignore')

# Load data
df: pd.DataFrame = utils.load_original_data()

### Model Building

### Spliting Data